In [ ]:
import random
import math
import uuid
from typing import List
import pandas as pd

# Constants
MATURE_AGE = 18
PROXIMITY_THRESHOLD = 2.0  # Distance within which reproduction is possible
SIMULATION_STEPS = 100
TIME_STEP = 1.0  # Each step is 1 unit of time
WORLD_SIZE = 100  # Boundary for the 2D plane

In [ ]:
# Helper function
def distance(p1, p2):
    return math.hypot(p1.x - p2.x, p1.y - p2.y)

In [ ]:
# Define person
class Person:
    def __init__(self, age, gender, sexual_pref, speed):
        self.id = uuid.uuid4()
        self.age = age
        self.gender = gender
        self.sexual_pref = sexual_pref  # "male", "female", or "any"
        self.speed = speed
        self.x = random.uniform(0, WORLD_SIZE)
        self.y = random.uniform(0, WORLD_SIZE)
        self.alive = True

    def move(self):
        angle = random.uniform(0, 2 * math.pi)
        dx = self.speed * math.cos(angle)
        dy = self.speed * math.sin(angle)
        self.x = max(0, min(WORLD_SIZE, self.x + dx))
        self.y = max(0, min(WORLD_SIZE, self.y + dy))

    def can_reproduce_with(self, other):
        return (
            self.age >= MATURE_AGE and
            other.age >= MATURE_AGE and
            self.id != other.id and
            (
                self.sexual_pref == "any" or self.sexual_pref == other.gender
            ) and (
                other.sexual_pref == "any" or other.sexual_pref == self.gender
            )
        )

In [ ]:
# Simulation class
class PopulationSimulation:
    def __init__(self, initial_population: int):
        self.people: List[Person] = [self.random_person() for _ in range(initial_population)]
        self.movements = []
        self.time = 0

    def random_person(self):
        return Person(
            age=random.randint(0, 40),
            gender=random.choice(["male", "female"]),
            sexual_pref=random.choice(["male", "female", "any"]),
            speed=random.uniform(0.5, 2.0)
        )

    def simulate(self, steps=SIMULATION_STEPS):
        for step in range(steps):
            self.time += TIME_STEP
            new_people = []
            for person in self.people:
                person.age += TIME_STEP / 365.0  # assume age is in years
                person.move()
                self.movements.append({
                    "time": self.time,
                    "id": str(person.id),
                    "age": person.age,
                    "x": person.x,
                    "y": person.y
                })

            # Check for reproduction opportunities
            for i in range(len(self.people)):
                for j in range(i + 1, len(self.people)):
                    p1 = self.people[i]
                    p2 = self.people[j]
                    if distance(p1, p2) < PROXIMITY_THRESHOLD and p1.can_reproduce_with(p2):
                        baby = self.random_person()
                        baby.x = (p1.x + p2.x) / 2
                        baby.y = (p1.y + p2.y) / 2
                        baby.age = 0
                        new_people.append(baby)
                        # Only allow one offspring per pair per step
                        break

            self.people.extend(new_people)

    def export_movements(self):
        return pd.DataFrame(self.movements)

In [ ]:
# Usage
sim = PopulationSimulation(initial_population=10)
sim.simulate(steps=50)
df_movements = sim.export_movements()

# Save to CSV if needed
df_movements.to_csv("population_movements.csv", index=False)

print(df_movements.head())